# 데이터 분석을 위한 추가 전처리
1. 'stos', 'dtos' 제거
2. 'label' 에서 'target 생성 및 label 제거
3. 숫자형 데이터 MinMaxScaling

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
from torch.utils.data import DataLoader,Dataset

from scipy.optimize import linear_sum_assignment as linear_assignment

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'{device} is available')

cuda:0 is available


In [ ]:
dataset=pd.read_csv('/content/drive/MyDrive/캡스톤/dataset/CTU-13(9)_preprocessed.csv')

In [ ]:
# column 명들 소문자로 변환
dataset.columns = dataset.columns.str.lower()

In [ ]:
# 'stos0' 'dtos0' 모두 1.0 의 값을 가지므로 분류에 필요하지 않은 것 같아 삭제 (9번 시나리오에서는)
dataset.drop(['stos0','dtos0'],axis=1,inplace=True)
dataset

,dur,proto0,proto1,proto2,proto3,sport0,sport1,sport2,sport3,sport4,...,dport3,dport4,state0,state1,state2,state3,totpkts,totbytes,srcbytes,label
0,1752.578735,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,467,274517,79210,flow=From-Normal-V50-Jist
1,0.000000,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1,130,130,flow=From-Normal-V50-UDP-CVUT-DNS-Server
2,1532.028076,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,71,4556,2154,flow=From-Normal-V50-Stribrek
3,3570.125732,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,117,7020,3480,flow=From-Normal-V50-Stribrek
4,3570.125000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,519,74059,19656,flow=From-Normal-V50-Stribrek
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214949,0.000282,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,2,400,74,flow=From-Normal-V50-Stribrek
214950,0.018888,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,6,412,272,flow=From-Normal-V50-Stribrek
214951,0.000378,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,2,244,81,flow=From-Normal-V50-Stribrek
214952,0.000319,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,2,280,81,flow=From-Normal-V50-Stribrek


In [ ]:
# Botnet 과 Normal 구분 -> 정상 0, 비정상 1
def convertLabel(sample):
  if 'Botnet' in sample: return 1
  else: return 0

dataset['target'] = dataset['label'].apply(convertLabel)

In [ ]:
# label feature 제거
dataset.drop(['label'],axis=1,inplace=True)

In [ ]:
# 숫자형 feature -> MinMaxScaling
columns=['dur','totpkts','totbytes','srcbytes']
scaler=MinMaxScaler()
dataset[columns]=scaler.fit_transform(dataset[columns])

In [ ]:
dataset

,dur,proto0,proto1,proto2,proto3,sport0,sport1,sport2,sport3,sport4,...,dport3,dport4,state0,state1,state2,state3,totpkts,totbytes,srcbytes,target
0,4.868403e-01,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.006758,3.996590e-03,0.001442,0
1,0.000000e+00,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.000000,9.902101e-07,0.000002,0
2,4.255746e-01,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.001015,6.544124e-05,0.000039,0
3,9.917278e-01,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.001682,1.013218e-04,0.000063,0
4,9.917276e-01,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.007512,1.077538e-03,0.000358,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214949,7.833540e-08,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.000015,4.921927e-06,0.000001,0
214950,5.246805e-06,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.000073,5.096670e-06,0.000005,0
214951,1.050028e-07,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.000015,2.650268e-06,0.000001,0
214952,8.861345e-08,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.000015,3.174497e-06,0.000001,0


## Train data, Test data 나누기
**논문에 나와있는 대로**
1. Train data -> Normal data(40%)
2. Test data -> Normal data(60%) + Abnormal data(60%)

In [ ]:
# 정상 dataset / 비정상 dataset
normal_dataset=dataset[dataset['target']==0]
abnormal_dataset=dataset[dataset['target']==1]

# 정상 dataset -> Train 정상 / Test 정상 나누기 , 40%:60%
normal_dataset = normal_dataset.drop(['target'],axis=1) # 'target' feature drop 
train_normal,test_normal=train_test_split(normal_dataset, test_size=0.6, random_state=42)

In [ ]:
test_normal

,dur,proto0,proto1,proto2,proto3,sport0,sport1,sport2,sport3,sport4,...,dport2,dport3,dport4,state0,state1,state2,state3,totpkts,totbytes,srcbytes
107126,9.950467e-01,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.004626,0.000604,0.000211
140268,3.319893e-05,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.000102,0.000025,0.000017
11906,8.305775e-08,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.000015,0.000005,0.000001
91241,9.278023e-07,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.000015,0.000002,0.000001
18125,1.875049e-07,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.000015,0.000003,0.000002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93781,9.250244e-08,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.000015,0.000002,0.000001
91616,1.477817e-07,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.000015,0.000003,0.000001
115560,2.553483e-03,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.000334,0.000161,0.000084
168990,8.645929e-02,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.003568,0.003126,0.000151


In [ ]:
# 비정상 dataset에서 random 으로 60% 추출
test_abnormal = abnormal_dataset.sample(frac=0.6)
test_abnormal = test_abnormal.drop(['target'],axis=1) # 'target' feature drop 

In [ ]:
test_abnormal

,dur,proto0,proto1,proto2,proto3,sport0,sport1,sport2,sport3,sport4,...,dport2,dport3,dport4,state0,state1,state2,state3,totpkts,totbytes,srcbytes
88524,4.629455e-05,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.000015,0.000003,0.000001
26100,1.129196e-05,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.000015,0.000002,0.000001
27446,2.503608e-03,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.000029,0.000002,0.000003
115733,0.000000e+00,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.000000,0.000000,0.000001
197058,4.875184e-05,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.000015,0.000002,0.000001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19436,1.000026e-07,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.000015,0.000005,0.000001
94834,7.412418e-06,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.000015,0.000004,0.000002
120771,4.657984e-05,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.000015,0.000003,0.000001
87688,0.000000e+00,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.000000,0.000000,0.000001


# Dataframe 을 pytorch tensor로 변환 

In [ ]:
# pytorch tensor 로 변환 

# determine the supported device
def get_device():
  if torch.cuda.is_available():
    device = torch.device('cuda:0')
  else :
    device = torch.device('cpu') # don 't have GPU 
  return device

# convert a df to tensor to be used in pytorch
def dataframe_to_tensor(df):
   device = get_device()
   return torch.from_numpy(df.values).float().to(device)

In [ ]:
train_normal = dataframe_to_tensor(train_normal)

# Deep K-means
1. layer 수는 5
2. mini-batch size 는 100 
3. 각 layer에서 얼만큼 줄일지 + 활성화 함수는 어떤 것을 사용할지 ?
4. learning rate=0.001 인데 어느 정도로 해야할지?

--> 3차원으로 latent vector 크기 잡아도 될듯

In [ ]:
# train_loader 생성
train_loader = DataLoader(train_normal,batch_size=100,shuffle=True)
train_batch = iter(train_loader).next()
print(train_batch.shape)


torch.Size([100, 27])


In [ ]:
class TestDataset(Dataset):

  def __init__(self,x_data,y_data):
    self.x_data = torch.FloatTensor(x_data)
    self.y_data = torch.LongTensor(y_data)
    self.len = len(self.x_data)

  def __getitem__(self,index):
    return self.x_data[index], self.y_data[index]

  def __len__(self):
    return self.len


In [ ]:
# test_loader 생성
test_data = pd.concat([test_normal, test_abnormal]).values
test_label = np.concatenate([np.zeros(17981),np.ones(110992)])

test_data = TestDataset(test_data,test_label)

test_loader = DataLoader(test_data,batch_size=100,shuffle=True)

In [ ]:
class Encoder(nn.Module):
  def __init__(self,latent_size):
    super(Encoder,self).__init__()

    self.encoder = nn.Sequential(
                      nn.Linear(27,16),
                      nn.ReLU(),
                      nn.Linear(16,latent_size),
                      nn.ReLU())
          
    
  def forward(self, x):
    return self.encoder(x)

class Decoder(nn.Module):
  def __init__(self, latent_size):
    super(Decoder,self).__init__()

    self.decoder = nn.Sequential(
                      nn.Linear(latent_size,16),
                      nn.ReLU(),
                      nn.Linear(16,27),
                      nn.Sigmoid())
    
  def forward(self,x):
    return self.decoder(x)

In [ ]:
class Kmeans(nn.Module):
  def __init__(self,num_clusters, latent_size):
    super(Kmeans,self).__init__()
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    self.num_clusters = num_clusters
    self.centroids = nn.Parameter(torch.rand((self.num_clusters,latent_size)).to(device))

  def argminl2distance(self,a,b):
    return torch.argmin(torch.sum((a-b)**2,dim=1),dim=0)

  def forward(self,x):
    y_assign = []
    for m in range(x.size(0)):
      h = x[m].expand(self.num_clusters,-1)
      assign = self.argminl2distance(h, self.centroids)
      y_assign.append(assign.item())
    return y_assign, self.centroids[y_assign]

In [ ]:
def cluster_acc(y_true,y_pred):
  y_true = np.array(y_true)
  y_pred = np.array(y_pred)
  D = max(y_pred.max(), y_true.max()) + 1
  w = np.zeros((D,D), dtype = np.int64)
  for i in range(y_pred.size):
    w[y_pred[i],y_true[i]] +=1
  ind = linear_assignment(w.max() - w)
  return sum([w[i,j] for i,j in zip(ind[0],ind[1])]) * 1.0 / y_pred.size

In [ ]:
def evaluation(testloader, encoder, kmeans, device):
  predictions = []
  actual = []

  with torch.no_grad():
    for images, labels in testloader:
      inputs = images.to(device)
      labels = labels.to(device)
      latent_var = encoder(inputs)
      y_pred,_ = kmeans(latent_var)

      predictions+=y_pred
      actual+=labels.cpu().tolist()

  return cluster_acc(actual,predictions)

#모델 학습


In [ ]:
latent_size=4
num_clusters=2

In [ ]:
encoder = Encoder(latent_size).to(device)
decoder = Decoder(latent_size).to(device)
kmeans = Kmeans(num_clusters, latent_size).to(device)
criterion1 = torch.nn.MSELoss()
criterion2 = torch.nn.MSELoss()
optimizer = torch.optim.Adam(list(encoder.parameters())+list(decoder.parameters())+list(kmeans.parameters()), lr=0.001)

In [ ]:
T1=100
T2=200
lam = 1e-3
ls = 0.05

In [ ]:
for ep in range(300):
  if ep <= T1:
    alpha = lam/(T2-T1)
  else:
    alpha = lam
  
  # 정상 data(label=0)에 대해서만 학습
  running_loss = 0.0
  for batch in train_loader:
    inputs = batch.to(device)
    optimizer.zero_grad()
    latent_var = encoder(inputs)
    _, centroids = kmeans(latent_var.detach())
    outputs = decoder(latent_var)

    l_rec = criterion1(inputs,outputs)
    l_clt = criterion2(latent_var,centroids)
    loss = l_rec + alpha*l_clt

    loss.backward()
    optimizer.step()
    running_loss+=loss.item()
  
  ###
  avg_loss = running_loss/len(train_loader)

  if(ep%10==0):
    print('[%d] Train loss:%.4f' %(ep,avg_loss) )

[0] Train loss:0.1985
[10] Train loss:0.0024
[20] Train loss:0.0012
[30] Train loss:0.0009
[40] Train loss:0.0007
[50] Train loss:0.0005
[60] Train loss:0.0005
[70] Train loss:0.0004
[80] Train loss:0.0003
[90] Train loss:0.0002
[100] Train loss:0.0002
[110] Train loss:0.0004
[120] Train loss:0.0003
[130] Train loss:0.0002
[140] Train loss:0.0002
[150] Train loss:0.0001
[160] Train loss:0.0001
[170] Train loss:0.0001
[180] Train loss:0.0001
[190] Train loss:0.0001
[200] Train loss:0.0001
[210] Train loss:0.0001
[220] Train loss:0.0001
[230] Train loss:0.0001
[240] Train loss:0.0001
[250] Train loss:0.0001
[260] Train loss:0.0001
[270] Train loss:0.0001
[280] Train loss:0.0001
[290] Train loss:0.0001


In [ ]:
torch.save(encoder.state_dict(),'/content/drive/MyDrive/캡스톤/models(CTU-13(9))/encoder.pth')
torch.save(decoder.state_dict(),'/content/drive/MyDrive/캡스톤/models(CTU-13(9))/decoder.pth')
torch.save(kmeans.state_dict(),'/content/drive/MyDrive/캡스톤/models(CTU-13(9))/kmeans.pth') 

# 학습 후 One class svm ensemble

In [ ]:
encoder.load_state_dict(torch.load('/content/drive/MyDrive/캡스톤/models(CTU-13(9))/encoder.pth'))
decoder.load_state_dict(torch.load('/content/drive/MyDrive/캡스톤/models(CTU-13(9))/decoder.pth'))
kmeans.load_state_dict(torch.load('/content/drive/MyDrive/캡스톤/models(CTU-13(9))/kmeans.pth'))

<All keys matched successfully>

In [ ]:
# svm 에 활용할 latent_vectors 생성
# svm 에 활용할 latent_vectors_labels 생성

train_latent_vectors = [] # latent vector 가 list로 계속해서 들어간 형태
train_predict_label= []

for data in train_loader:
  inputs = data.to(device)
  latent_vector= encoder(inputs)
  predict_label, _ = kmeans(latent_vector)

  train_latent_vectors+=latent_vector.cpu().tolist()
  train_predict_label+=predict_label


In [ ]:
# svm 에 활용할 test_latent_vectors 생성
# svm 에 활용할 test_label (실제 label 값) 생성

test_latent_vectors = [] # latent vector 가 list로 계속해서 들어간 형태
test_label= []

for data,labels in test_loader:
  inputs = data.to(device)
  latent_vector= encoder(inputs)

  test_latent_vectors+=latent_vector.cpu().tolist()
  test_label+=labels.cpu().tolist()

# OneSvm Ensemble

In [ ]:
#OneclassSVM Ensemble
from sklearn.svm import OneClassSVM
import numpy as np
import pandas as pd

class OCSVMEnsemble():

  def __init__(self,nu=0.5):
    self.nu = nu
    #self.gamma = gamma

  def fit(self, latent, pred):
    # oneclasssvm 인스턴스가 들어갈 리스트
    self.instance = []
    self.num_cluster = len(np.unique(pred))

    # 경계가 9개 생긴거
    for clu in range(self.num_cluster):
      idx = np.where(pred == clu) # 0-9 까지의 각 그룹에 해당하는 index 도출 
      #해당 군집에 속한 data
      clu_data = latent[idx]
      ocsvm = OneClassSVM(kernel = 'rbf',gamma = 'scale', nu = self.nu).fit(clu_data) # OneClassSVM 을 통한 객체 ocsvm 생성
      self.instance.append(ocsvm)

  # 모든 OneClassSVM 인스턴스를 돌면서
  # inlier -> 0 , outlier -> 1 return 하는 함수
  def predictLabel(self,x):
    for model in self.instance:
      model_predict = model.predict(x) # inliers 1, outliers -1
      if model_predict == 1:
        return 0
    return 1


In [ ]:
# train_latent_vectors, latent_vectors_labels numpy 로 변환
train_latent_vectors=np.array(train_latent_vectors)
train_predict_label=np.array(train_predict_label)

In [ ]:
# test_latent_vectors, test_label numpy 로 변환
test_latent_vectors = np.array(test_latent_vectors)
test_label = np.array(test_label)

In [ ]:
# 학습 과정
ocsvm = OCSVMEnsemble(nu=0.01) 
ocsvm.fit(train_latent_vectors, train_predict_label) # train data 에서 latent vector 들 + 가장 가까운 중심의 index

In [ ]:
train_latent_vectors[0:10]

array([[4.72243166, 6.29423666, 0.        , 0.26086637],
       [4.7592144 , 6.30096292, 0.        , 0.        ],
       [4.72243118, 6.29423666, 0.        , 0.26086619],
       [4.72235107, 6.29413939, 0.        , 0.26087147],
       [4.57197285, 6.01034594, 0.        , 0.        ],
       [5.0145483 , 6.15556526, 0.        , 0.        ],
       [0.93448198, 3.55725646, 0.        , 0.        ],
       [4.74920988, 6.28541613, 0.        , 0.        ],
       [4.72243261, 6.29423714, 0.        , 0.26086637],
       [4.72243309, 6.29423714, 0.        , 0.26086661]])

In [ ]:
train_predict_label[0:10]

array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0])

In [ ]:
test_latent_vectors[20:30]

array([[4.75384665, 6.29258776, 0.        , 0.        ],
       [4.72240925, 6.29421616, 0.        , 0.26086739],
       [4.72242928, 6.29423618, 0.        , 0.26086649],
       [4.72243023, 6.29423666, 0.        , 0.26086646],
       [4.72242641, 6.29423475, 0.        , 0.26086611],
       [4.72243261, 6.29423714, 0.        , 0.26086637],
       [4.9793663 , 6.09503508, 0.        , 0.        ],
       [4.7224288 , 6.29423475, 0.        , 0.26086602],
       [4.72239971, 6.29419994, 0.        , 0.2608681 ],
       [4.72242928, 6.29423618, 0.        , 0.26086649]])

In [ ]:
test_label[20:30]

array([1, 1, 1, 1, 1, 1, 0, 1, 1, 1])

In [ ]:
kmeans.centroids

Parameter containing:
tensor([[ 4.8324e+00,  6.2235e+00, -2.6840e-05,  1.6824e-01],
        [ 1.3298e+00,  3.0066e+00, -2.6804e-41,  1.3972e-01]], device='cuda:0',
       requires_grad=True)

In [ ]:
# test과정에서의 latent vector 들을 넣어가면서 예측하는 과정
y_pred = []
for i in test_latent_vectors:
  i=i.reshape(1,-1)
  pred=ocsvm.predictLabel(i)
  y_pred.append(pred)

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
print(classification_report(test_label,y_pred))

              precision    recall  f1-score   support

           0       0.15      0.66      0.24     17981
           1       0.87      0.38      0.53    110992

    accuracy                           0.42    128973
   macro avg       0.51      0.52      0.39    128973
weighted avg       0.77      0.42      0.49    128973



In [ ]:
y_pred[20:30]

[0, 1, 1, 1, 1, 1, 0, 1, 1, 1]